In [88]:
%%time
import json
import numpy as np
import os
import pdb
import requests
import pandas as pd
from IPython.display import clear_output

CPU times: user 42 μs, sys: 4 μs, total: 46 μs
Wall time: 58.4 μs


In [89]:
%%time
# both input4MIPs and input4mips
actId = "input4MIPs"
projId = "input4MIPs"
mip_era = "CMIP7"


CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.96 μs


In [90]:
url = "http://esgf-node.llnl.gov/esg-search/search?format=application%2Fsolr%2Bjson&activity_id="+actId+"&project="+projId+"&mip_era="+mip_era+"&type=Dataset"
#js=requests.get(inputsD)
#js_mipsD = json.loads(js.text)

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    
    # Accessing response data
    print("Status Code:", response.status_code)
    print("Headers:", response.headers)
    
    # Attempt to parse JSON if possible, otherwise return raw text
    try:
        data = response.json()
        print("JSON Data:", data)
    except ValueError:
        print("Raw Response Text:", response.text)
        
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

Status Code: 200
Headers: {'Date': 'Sun, 23 Feb 2025 16:37:42 GMT', 'Server': 'Apache', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-XSS-Protection': '1; mode=block', 'X-Frame-Options': 'DENY', 'X-Content-Type-Options': 'nosniff', 'Content-Type': 'text/json;charset=ISO-8859-1', 'Connection': 'close', 'Transfer-Encoding': 'chunked'}
JSON Data: {'responseHeader': {'status': 0, 'QTime': 6, 'params': {'df': 'text', 'q.alt': '*:*', 'indent': 'true', 'echoParams': 'all', 'fl': '*,score', 'start': '0', 'fq': ['type:Dataset', 'activity_id:"input4MIPs"', 'project:"input4MIPs"', 'mip_era:"CMIP7"'], 'rows': '10', 'q': '*:*', 'shards': 'localhost:8983/solr/datasets,localhost:8985/solr/datasets,localhost:8987/solr/datasets,localhost:8988/solr/datasets,localhost:8990/solr/datasets,localhost:8993/solr/datasets,localhost:8995/solr/datasets', 'tie': '0.01', 'facet.limit': '2048', 'qf': 'text', 'facet.method': 'fc', 'facet.mincount': '1', 'w

In [91]:
data=json.loads(response.text)
type(data)#["source_id"]
data.keys()
data['responseHeader']
dlist = pd.DataFrame(data['response']['docs'])


In [99]:
dlist[['source_id','variable', 'contact', 'title']]

,source_id,variable,contact,title
0,[SOLARIS-HEPPA-CMIP-4-6],[Multiple],[bernd@iaa.es],input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HE...
1,[SOLARIS-HEPPA-CMIP-4-6],[Multiple],[bernd@iaa.es],input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HE...
2,[SOLARIS-HEPPA-CMIP-4-6],[Multiple],[bernd@iaa.es],input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HE...


In [96]:
for title in dlist['title']:
    print(title)

input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HEPPA-CMIP-4-6.atmos.day.multiple.gn
input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HEPPA-CMIP-4-6.atmos.fx.multiple.gn
input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HEPPA-CMIP-4-6.atmos.mon.multiple.gn


In [97]:
dlist.keys() # 

Index(['project', 'activity_id', 'mip_era', 'target_mip', 'institution_id',
       'source_id', 'realm', 'frequency', 'variable_id', 'grid_label',
       'contact', 'dataset_category', 'source_version', 'source',
       'further_info_url', 'title', 'product', 'nominal_resolution',
       'deprecated', 'dataset_status', 'Conventions', 'target_mip_list',
       'creation_date', 'data_node', 'index_node', 'master_id', 'instance_id',
       'id', 'short_description', 'replica', 'latest', 'type', 'version',
       'dataset_id_template_', 'directory_format_template_',
       'variable_long_name', 'variable', 'variable_units', 'cf_standard_name',
       'datetime_start', 'datetime_end', 'height_top', 'height_bottom',
       'geo_units', 'number_of_files', 'size', 'access', 'globus_url', 'pid',
       'xlink', '_version_', 'retracted', '_timestamp', 'score'],
      dtype='object')

In [142]:
#https://aims2.llnl.gov/search/input4mips/?mip_era=CMIP7&target_mip=CMIP&activity_id=input4MIPs&institution_id=SOLARIS-HEPPA&source_id=SOLARIS-HEPPA-CMIP-4-6&frequency=day

def assemble_doi_inputs(df):
    # assemble title
    title = df["title"]
    print(f"Title: {df['title']}")

    # assemble description 
    description = f"""{df['mip_era'][0].replace("'", "")} Forcing Datasets ({df['activity_id'][0].replace("'", "")}).
    These data include all datasets published for '{df['title'].replace("'", "")}' with the full Data Reference Syntax following the template 'activity_id.mip_era.target_mip.institution_id.source_id.realm.frequency.variable_id.grid_label'.

    The model {df['source_id'][0].replace("'", "")} was run by the {df['institution_id'][0].replace("'", "")} in native nominal resolutions: unknown.

    Project: The forcing datasets (and boundary conditions) needed for {df['mip_era'][0].replace("'", "")} experiments are being prepared by a number of different experts. Initially many of these datasets may only be available from those experts, but over time as part of the 'input4MIPs' activity most of them will be archived by PCMDI and served by the Earth System Grid Federation (https://esgf-node.llnl.gov/search/input4mips/ ). More information is available in the living document: http://goo.gl/r8up31 .
    """
    print(description)
    
    # assemble URL
    ESGF_URL=f"https://aims2.llnl.gov/search/input4mips/?mip_era={df['mip_era'][0].replace("'", "")}&target_mip={df['target_mip'][0].replace("'", "")}&activity_id=x{df['activity_id'][0].replace("'", "")}&institution_id={df['institution_id'][0].replace("'", "")}&source_id={df['source_id'][0].replace("'", "")}&frequency={df['frequency'][0].replace("'", "")}"
    print(ESGF_URL)
    # test if ESGF_URL is valid
    try:
        page = requests.get(ESGF_URL)
        if page.status_code == 200:
            print(f"{page.status_code}: URL accessible.")
    except (requests.exceptions.HTTPError, requests.exceptions.ConnectionError):
        print("Error")

for i, df in dlist.iterrows():
    assemble_doi_inputs(df)

Title: input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HEPPA-CMIP-4-6.atmos.day.multiple.gn
CMIP7 Forcing Datasets (input4MIPs).
    These data include all datasets published for 'input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HEPPA-CMIP-4-6.atmos.day.multiple.gn' with the full Data Reference Syntax following the template 'activity_id.mip_era.target_mip.institution_id.source_id.realm.frequency.variable_id.grid_label'.

    The model SOLARIS-HEPPA-CMIP-4-6 was run by the SOLARIS-HEPPA in native nominal resolutions: unknown.

    Project: The forcing datasets (and boundary conditions) needed for CMIP7 experiments are being prepared by a number of different experts. Initially many of these datasets may only be available from those experts, but over time as part of the 'input4MIPs' activity most of them will be archived by PCMDI and served by the Earth System Grid Federation (https://esgf-node.llnl.gov/search/input4mips/ ). More information is available in the living document: http://goo.gl/r8up3

In [112]:
for i,df in dlist.iterrows():
    print(type(df))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [136]:
df["title"]#.replace("'", "")

'input4MIPs.CMIP7.CMIP.SOLARIS-HEPPA.SOLARIS-HEPPA-CMIP-4-6.atmos.mon.multiple.gn'